In [46]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [4]:
#read in race incidence csv from github

incidence_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Race_Incidence_Cancer_DB.csv")                         
#get rid of commas in columns
incidence_df = incidence_df.replace(',','', regex=True)
incidence_df.head()     

,average_annual_count,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,county_state,index_of_medical_underservice_score,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019
0,30,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,681.2,Lowndes County AL,54.0,10350,10248,10097,9974,9726
1,30,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,694.0,Lowndes County AL,54.0,10350,10248,10097,9974,9726
2,51,Alabama,All Cancer Sites,Black (includes Hispanic),All Sexes,All Ages,570.1,Lowndes County AL,54.0,10350,10248,10097,9974,9726
3,80,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,558.3,Crenshaw County AL,58.0,13850,13916,13858,13825,13772
4,80,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,565.3,Crenshaw County AL,58.0,13850,13916,13858,13825,13772


In [5]:
#get count of rows
incidence_df.count()

average_annual_count                   9160
primary_state_name                     9160
cancer                                 9160
race_ethnicity                         9160
sex                                    9160
age                                    9160
age_adjusted_rate                      9160
county_state                           9160
index_of_medical_underservice_score    7931
popestimate2015                        9160
popestimate2016                        9160
popestimate2017                        9160
popestimate2018                        9160
popestimate2019                        9160
dtype: int64

In [7]:
#get count of values for race
incidence_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2643
  White Non-Hispanic                            2335
Black (includes Hispanic)                       1308
Hispanic (any race)                             1009
  White Hispanic                                 912
Asian / Pacific Islander (includes Hispanic)     525
Amer. Indian / AK Native (includes Hispanic)     428
Name: race_ethnicity, dtype: int64

In [8]:
#get count of NA values for MUA
incidence_df["index_of_medical_underservice_score"].isna().sum()

1229

In [9]:
#drop rows where MUA index is null
incidence_df = incidence_df[incidence_df["index_of_medical_underservice_score"].isna() == False]

In [10]:
#create target 
y = incidence_df[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
0,681.2
1,694.0
2,570.1
3,558.3
4,565.3


In [11]:
#create feautures variable holding ages
X = incidence_df[["race_ethnicity", "index_of_medical_underservice_score"]]
X.head()

,race_ethnicity,index_of_medical_underservice_score
0,White (includes Hispanic),54.0
1,White Non-Hispanic,54.0
2,Black (includes Hispanic),54.0
3,White (includes Hispanic),58.0
4,White Non-Hispanic,58.0


In [12]:
#create dummy variables for age
X = pd.get_dummies(X, columns=["race_ethnicity"])
X.head()

,index_of_medical_underservice_score,race_ethnicity_Amer. Indian / AK Native (includes Hispanic),race_ethnicity_Asian / Pacific Islander (includes Hispanic),race_ethnicity_Black (includes Hispanic),race_ethnicity_Hispanic (any race),race_ethnicity_White (includes Hispanic),race_ethnicity_ White Hispanic,race_ethnicity_ White Non-Hispanic
0,54.0,0,0,0,0,1,0,0
1,54.0,0,0,0,0,0,0,1
2,54.0,0,0,1,0,0,0,0
3,58.0,0,0,0,0,1,0,0
4,58.0,0,0,0,0,0,0,1


In [13]:
#get shape of target array
y.shape

(7931, 1)

In [14]:
#get shape of feautures array
X.shape

(7931, 8)

In [15]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [16]:
#create linear regression model using sklearn
model = LinearRegression()

In [17]:
#fit the model
regression_race_MUA_incidence = model.fit(X_train ,y_train)

In [19]:
#Create predictions array using linear regression
y_pred = regression_race_MUA_incidence.predict(X_test)
y_pred

array([[424.57808079],
       [450.15734848],
       [451.10011881],
       ...,
       [452.04288915],
       [451.10011881],
       [449.69488993]])

In [20]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.2778661304937293

In [33]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [40]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [41]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     342.0
Date:                Sun, 29 May 2022   Prob (F-statistic):               0.00
Time:                        14:13:45   Log-Likelihood:                -47692.
No. Observations:                7931   AIC:                         9.540e+04
Df Residuals:                    7923   BIC:                         9.546e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [42]:
##decision tree model
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [43]:
#fit the model with training data
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [44]:
#predict
y_pred = regressor_DT.predict(X)
y_pred

array([436.35      , 436.45142857, 443.5       , ..., 388.15      ,
       452.03618421, 455.83030303])

In [45]:

#calculate r2 score
r2_score(y, y_pred)

0.31184415741166505